In [0]:
%pip install snowflake-connector-python

In [0]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [0]:
sourceFileUrl="/Workspace/Users/databrickslearning30days@gmail.com/AZURE-Databricks/Bronze/BronzeIngest.csv"
df = pd.read_csv(sourceFileUrl)

In [0]:
conn = snowflake.connector.connect(
    account="HDKHNTT-IP84963",
    user="HITLER",
    password="KumarG@urav234892",
    warehouse="SNOWFLAKE_LEARNING_WH",
    database="DAILY_PRICING",
    schema="PUBLIC"
)
cur = conn.cursor()
cur.execute("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA()")
print(cur.fetchone())
cur.close()
conn.close()

In [0]:
from snowflake.connector import connect

with connect(
    account="HDKHNTT-IP84963",
    user="HITLER",
    password="KumarG@urav234892",
    warehouse="SNOWFLAKE_LEARNING_WH",
    database="DAILY_PRICING",
    schema="PUBLIC"
) as conn:
    with conn.cursor() as cur:
        cur.execute("SHOW WAREHOUSES")
        print([r[0] for r in cur.fetchall()])

        cur.execute("SHOW DATABASES")
        print([r[1] for r in cur.fetchall()])

        cur.execute("SHOW SCHEMAS IN DATABASE DAILY_PRICING")
        print([r[1] for r in cur.fetchall()])

In [0]:
from snowflake.connector import connect

with connect(
    account="HDKHNTT-IP84963",
    user="HITLER",
    password="KumarG@urav234892",
    warehouse="SNOWFLAKE_LEARNING_WH",
    database="DAILY_PRICING",
    schema="PUBLIC"
) as conn:
    with conn.cursor() as cur:
        cur.execute("USE WAREHOUSE SNOWFLAKE_LEARNING_WH")
        cur.execute("USE DATABASE DAILY_PRICING")
        cur.execute("USE SCHEMA PUBLIC")

In [0]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

pdf = pd.read_csv(sourceFileUrl)

# Identify date columns (adjust this list as needed)
date_cols = [col for col in pdf.columns if "date" in col.lower()]

# Convert date columns to datetime, coerce errors to NaT
for col in date_cols:
    pdf[col] = pd.to_datetime(pdf[col], errors="coerce").astype(str)

# For non-date object columns, convert to numeric
for col in pdf.select_dtypes(include=["object"]).columns:
    if col not in date_cols:
        pdf[col] = pd.to_numeric(pdf[col], errors="coerce")

# Fill NaN in date columns with a default date
for col in date_cols:
    pdf[col] = pdf[col].fillna(pd.Timestamp("1970-01-01"))

# Fill NaN in other columns with 0
pdf = pdf.fillna(0)

conn = snowflake.connector.connect(
    account="HDKHNTT-IP84963",
    user="HITLER",
    password="KumarG@urav234892",
    warehouse="SNOWFLAKE_LEARNING_WH",
    database="SNOWFLAKE_LEARNING_DB",
    schema="HITLER_LOAD_SAMPLE_DATA_FROM_S3"
)

success, nchunks, nrows, _ = write_pandas(conn, pdf, "DAILY_PRICING")
print("loaded:", success, "rows:", nrows)

conn.close()